In [18]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv('EWSGSE967.txt')
data = data.rename(columns = {'class': 'Class'})

In [20]:
data = data.drop(data.index[[0,1]])
data = data.drop(['sample'], axis=1)
data.index=range(0,len(data),1)
data.head()

,31307_at,31309_r_at,31313_at,31314_at,31315_at,31316_at,31317_r_at,31320_at,31321_at,31322_at,31323_r_at,31324_at,31328_at,31329_at,31330_at,31331_at,31333_at,31335_at,31339_at,31344_at,31345_at,31349_at,31350_at,31352_at,31353_f_at,31356_at,31357_at,31359_at,31363_at,31364_i_at,31365_f_at,31371_at,31372_at,31373_at,31376_at,31378_at,31379_at,31383_at,31384_at,31385_at,...,141_s_at,142_at,146_at,147_at,148_at,149_at,150_at,151_s_at,153_f_at,154_at,113_i_at,114_r_at,116_at,117_at,119_at,120_at,121_at,124_i_at,125_r_at,126_s_at,127_at,128_at,129_g_at,131_at,132_at,133_at,134_at,135_g_at,111_at,100_g_at,101_at,102_at,103_at,104_at,105_at,106_at,107_at,108_g_at,109_at,Class
0,10,10,206.7,217.3,95,1590.1,10,1112.9,118.2,10,725.7,10,62,92590.8,10,254.8,264.2,10,10,482.7,151,131,10,1795.2,10,145.2,176.8,10,10,342.4,10,1204.9,10,10.8,86.1,53.7,53.3,10,35709,764.6,...,10,570.5,1743.5,908.1,1694.4,124.3,100697.8,10,10,10,10,530.3,59.7,10,657.7,4162.8,10,10,58.2,10,830.6,10,10,10,416.7,10,10,1296.7,387.6,10,226.2,414.8,672.9,10,5141.4,57.9,10,2595.3,1340.2,EWS
1,10,10,10,10,289.1,10,2350.7,207.4,456.3,402.9,10,412.8,386.2,10,157825.8,10,348.1,294.3,10,610.6,86.8,10,10,243.4,897.3,581.1,310.1,10,21.3,10,168.2,10,891.6,10,29.4,10,343,10,10,115624.3,...,10,10,10,1918.6,1007.7,1991.1,275.2,58563.3,374.4,10,447.5,10,493.6,10,10,1685.9,5417.5,10,10,73.1,10,16.5,10,611.8,10,802.9,626.6,230.7,1247.7,950.5,1597.2,315.3,1458,10,530.7,1618.1,55.5,567.4,2517.4,EWS
2,10,10,10,10,606.2,316.1,1061.9,10,488.2,98.1,110,10,260.2,322.9,110835.7,34.2,10,10,198.8,820.8,275.4,260.2,10,425.5,964.7,646.7,588.2,10,204.2,10,333.1,10,246.4,206.2,418.3,10,104.5,10,10,61946.8,...,10,10,10,1392.3,676.9,644.9,702.8,68256.4,318.1,51.6,134.4,332.2,160.7,10,10,951.4,4231.3,10,10,454.5,10,2407.7,1212,934.6,10,1126.3,119.7,10,748,649.1,318.6,814.4,1189.7,97.9,431.2,2856.7,10,10,3572.6,EWS
3,10,124,10,174.7,331.8,253.3,1227.3,372.7,462.1,10,10,597.3,292.5,119.3,114888.1,206.2,10,10,10,10,25.7,10,10,10,1449.3,431.8,336.7,160.8,328.8,234.8,380.5,316.5,1089.8,10,25.1,147.9,307.9,46.4,10,56772.1,...,10,10,163.8,1624.8,955.6,1106.2,81.6,116916,482.9,286.4,10,203.7,47.6,337.1,10,617.7,5301.4,10,10,329.3,10,10,234.5,10,26.4,320.4,10,235.5,687.7,1025.2,734.5,10,1692,338.4,10,7958.5,481.1,10,2695.1,EWS
4,10,10,10,74.9,10,10,1719.8,10,748.5,245.7,140.6,1111.1,909.5,602.9,99968.9,149.5,542.6,10,10,592.3,390.8,10,10,10,1066.6,654.2,10,319.6,78.4,267.3,475.5,10,1592.5,252.3,10,10,617.6,10,10,12625.8,...,1915.1,10,10,2189.1,617.5,10,10,51164.2,10,10,664.1,10,23.4,56.2,10,840.1,5000.2,10,88.6,205.3,10,322.5,10,204.5,145.4,1042.7,32.1,10,10,535.2,10,403.5,2527.6,518.1,10,1772.2,933.2,10,1754.4,EWS


In [21]:
data.dropna(inplace=True)

Dict e Class chilo na, class chilo, to replace korchilo na dict e 

In [22]:
dict = {
    'Class':{
        'EWS':0,
        'RMS':1
    }
}   
data=data.replace(dict)
data.head()


,31307_at,31309_r_at,31313_at,31314_at,31315_at,31316_at,31317_r_at,31320_at,31321_at,31322_at,31323_r_at,31324_at,31328_at,31329_at,31330_at,31331_at,31333_at,31335_at,31339_at,31344_at,31345_at,31349_at,31350_at,31352_at,31353_f_at,31356_at,31357_at,31359_at,31363_at,31364_i_at,31365_f_at,31371_at,31372_at,31373_at,31376_at,31378_at,31379_at,31383_at,31384_at,31385_at,...,141_s_at,142_at,146_at,147_at,148_at,149_at,150_at,151_s_at,153_f_at,154_at,113_i_at,114_r_at,116_at,117_at,119_at,120_at,121_at,124_i_at,125_r_at,126_s_at,127_at,128_at,129_g_at,131_at,132_at,133_at,134_at,135_g_at,111_at,100_g_at,101_at,102_at,103_at,104_at,105_at,106_at,107_at,108_g_at,109_at,Class
0,10,10,206.7,217.3,95,1590.1,10,1112.9,118.2,10,725.7,10,62,92590.8,10,254.8,264.2,10,10,482.7,151,131,10,1795.2,10,145.2,176.8,10,10,342.4,10,1204.9,10,10.8,86.1,53.7,53.3,10,35709,764.6,...,10,570.5,1743.5,908.1,1694.4,124.3,100697.8,10,10,10,10,530.3,59.7,10,657.7,4162.8,10,10,58.2,10,830.6,10,10,10,416.7,10,10,1296.7,387.6,10,226.2,414.8,672.9,10,5141.4,57.9,10,2595.3,1340.2,0
1,10,10,10,10,289.1,10,2350.7,207.4,456.3,402.9,10,412.8,386.2,10,157825.8,10,348.1,294.3,10,610.6,86.8,10,10,243.4,897.3,581.1,310.1,10,21.3,10,168.2,10,891.6,10,29.4,10,343,10,10,115624.3,...,10,10,10,1918.6,1007.7,1991.1,275.2,58563.3,374.4,10,447.5,10,493.6,10,10,1685.9,5417.5,10,10,73.1,10,16.5,10,611.8,10,802.9,626.6,230.7,1247.7,950.5,1597.2,315.3,1458,10,530.7,1618.1,55.5,567.4,2517.4,0
2,10,10,10,10,606.2,316.1,1061.9,10,488.2,98.1,110,10,260.2,322.9,110835.7,34.2,10,10,198.8,820.8,275.4,260.2,10,425.5,964.7,646.7,588.2,10,204.2,10,333.1,10,246.4,206.2,418.3,10,104.5,10,10,61946.8,...,10,10,10,1392.3,676.9,644.9,702.8,68256.4,318.1,51.6,134.4,332.2,160.7,10,10,951.4,4231.3,10,10,454.5,10,2407.7,1212,934.6,10,1126.3,119.7,10,748,649.1,318.6,814.4,1189.7,97.9,431.2,2856.7,10,10,3572.6,0
3,10,124,10,174.7,331.8,253.3,1227.3,372.7,462.1,10,10,597.3,292.5,119.3,114888.1,206.2,10,10,10,10,25.7,10,10,10,1449.3,431.8,336.7,160.8,328.8,234.8,380.5,316.5,1089.8,10,25.1,147.9,307.9,46.4,10,56772.1,...,10,10,163.8,1624.8,955.6,1106.2,81.6,116916,482.9,286.4,10,203.7,47.6,337.1,10,617.7,5301.4,10,10,329.3,10,10,234.5,10,26.4,320.4,10,235.5,687.7,1025.2,734.5,10,1692,338.4,10,7958.5,481.1,10,2695.1,0
4,10,10,10,74.9,10,10,1719.8,10,748.5,245.7,140.6,1111.1,909.5,602.9,99968.9,149.5,542.6,10,10,592.3,390.8,10,10,10,1066.6,654.2,10,319.6,78.4,267.3,475.5,10,1592.5,252.3,10,10,617.6,10,10,12625.8,...,1915.1,10,10,2189.1,617.5,10,10,51164.2,10,10,664.1,10,23.4,56.2,10,840.1,5000.2,10,88.6,205.3,10,322.5,10,204.5,145.4,1042.7,32.1,10,10,535.2,10,403.5,2527.6,518.1,10,1772.2,933.2,10,1754.4,0


In [23]:
from sklearn.model_selection import train_test_split
X = data.drop(['Class'], axis=1)
y = data.Class
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [24]:
from sklearn.svm import SVC, LinearSVC
SVM = SVC()
SVM.fit(x_train, y_train)
SVM.score(x_test,y_test)

0.4

In [25]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(x_train, y_train)
KNN.score(x_test,y_test)

0.6

In [26]:
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
NB.fit(x_train, y_train)
NB.score(x_test,y_test)

0.8

In [27]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(x_train, y_train)
DT.score(x_test,y_test)

0.8

In [28]:
#Chi-Square Test

import scipy.stats as stats
from scipy.stats import chi2

def chi_sqr(test_column,output_column):
    observed_value = pd.crosstab(test_column,output_column)
    #print('observed value:\n',observed_value)
    val = stats.chi2_contingency(observed_value)
    expected_value=val[3]
    #print('expected value:\n',expected_value)
    no_of_rows=len(observed_value.iloc[0:2,0])
    no_of_columns=len(observed_value.iloc[0,0:2])
    dgof=(no_of_rows-1)*(no_of_columns-1)
    alpha=0.05
    #print('degree of freedom:',dgof)
    for o,e in zip(observed_value.values,expected_value):
        chi_2 = sum([(o-e)**2./e])       
    chi2_stat = np.sum(chi_2)
    return chi2_stat
    #critical_value = chi2.ppf(q=1-alpha, df=dgof)
    #print('critical_value:',critical_value)
    
    #p_value=1-chi2.cdf(x=chi2_stat, df=dgof)
    #print(p_value)

In [30]:
df1 = data.drop(['Class'], axis=1)
top_n1 =[]
for i in df1.columns.values:
    top_n1.append(chi_sqr(df1[i],data['Class']))

In [31]:
col_name1 = np.array(X.columns)
a1 = pd.DataFrame(top_n1)
b1 = pd.DataFrame(col_name1)
info1 = pd.concat([a1,b1], axis=1)
info1.columns = ['Score','Features']
info1

,Score,Features
0,0.916667,31307_at
1,1.090909,31309_r_at
2,0.916667,31313_at
3,1.090909,31314_at
4,1.090909,31315_at
...,...,...
9940,0.916667,105_at
9941,0.916667,106_at
9942,1.090909,107_at
9943,1.090909,108_g_at


In [32]:
top1 = info1.nsmallest(50,'Score')
ft1 = np.array(top1['Features'])
top1
ft1

array(['31307_at', '31313_at', '31317_r_at', '31320_at', '31323_r_at',
       '31324_at', '31331_at', '31335_at', '31344_at', '31345_at',
       '31349_at', '31350_at', '31352_at', '31353_f_at', '31356_at',
       '31359_at', '31363_at', '31365_f_at', '31372_at', '31373_at',
       '31385_at', '31386_at', '31390_at', '31397_at', '31403_at',
       '31413_at', '31414_at', '31417_at', '31418_at', '31421_at',
       '31422_at', '31424_at', '31425_g_at', '31429_at', '31430_at',
       '31437_r_at', '31439_f_at', '31440_at', '31447_at', '31448_s_at',
       '31449_at', '31452_at', '31454_f_at', '31455_r_at', '31462_f_at',
       '31471_at', '31473_s_at', '31474_r_at', '31478_at', '31480_f_at'],
      dtype=object)

In [33]:
#Information Gain/Mutual Information

from sklearn.feature_selection import mutual_info_classif

res = mutual_info_classif(X,y,discrete_features=True)

col_name2 = np.array(X.columns)

a2 = pd.DataFrame(res)
b2 = pd.DataFrame(col_name2)
info2 = pd.concat([a2,b2], axis=1)
info2.columns = ['Info_Gain','Features']
info2

,Info_Gain,Features
0,0.253281,31307_at
1,0.378710,31309_r_at
2,0.120747,31313_at
3,0.423391,31314_at
4,0.378710,31315_at
...,...,...
9940,0.302097,105_at
9941,0.692202,106_at
9942,0.510120,107_at
9943,0.215461,108_g_at


In [34]:
top2 = info2.nlargest(50,'Info_Gain')
ft2 = np.array(top2['Features'])
ft2

array(['38576_at', '38836_at', '33516_at', '39271_at', '32134_at',
       '37539_at', '33605_at', '34640_at', '36703_at', '37161_at',
       '39642_at', '41371_at', '41621_i_at', '31790_at', '32044_at',
       '33292_at', '33785_at', '34198_at', '34269_at', '36065_at',
       '36818_at', '37577_at', '38273_at', '38276_at', '38647_at',
       '38717_at', '39052_at', '39752_at', '40133_s_at', '40145_at',
       '40785_g_at', '40805_at', '40817_at', '40837_at', '41131_f_at',
       '41138_at', '41236_at', '32753_at', '33434_at', '33445_at',
       '33835_at', '33900_at', '33906_at', '33912_at', '34312_at',
       '34363_at', '34857_at', '34886_at', '35285_at', '36120_at'],
      dtype=object)

In [36]:
pip install skrebate

  Created wheel for skrebate: filename=skrebate-0.61-cp36-none-any.whl size=29259 sha256=5770e7bea0e53c39ebcd3ee969a8256124b0b297ffa31cf30a0a8b1e7e755b39
  Stored in directory: /root/.cache/pip/wheels/ae/d8/ae/9b51d487e9d02219996d6c260255a216ef07d905b0a0b00ce3
Successfully built skrebate


In [37]:
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF

features, classes = data.drop('Class', axis=1).values, data['Class'].values
X_train, X_test, y_train, y_test = train_test_split(features, classes)

arr = X_train.astype('float64')
fs = ReliefF()
fs.fit(arr, y_train)

top_n2=[]
for feature_name, feature_score in zip(data.drop('Class', axis=1).columns, fs.feature_importances_):
    top_n2.append(feature_score)

In [38]:
col_name3 = np.array(data.columns)
a3 = pd.DataFrame(top_n2)
b3 = pd.DataFrame(col_name3)
info3 = pd.concat([a3,b3], axis=1)
info3.columns = ['Score','Features']
info3

,Score,Features
0,0.144608,31307_at
1,0.107843,31309_r_at
2,0.029412,31313_at
3,0.084150,31314_at
4,0.011826,31315_at
...,...,...
9941,0.104233,106_at
9942,0.043179,107_at
9943,0.048203,108_g_at
9944,0.049166,109_at


In [39]:
top3 = info3.nlargest(50,'Score')
ft3 = np.array(top3['Features'])
ft3

array(['40485_at', '39133_at', '41138_at', '35330_at', '36561_at',
       '32144_at', '32756_at', '36943_r_at', '256_s_at', '39542_at',
       '38518_at', '41215_s_at', '35370_at', '35426_at', '36118_at',
       '34592_at', '33929_at', '39817_s_at', '40098_at', '35816_at',
       '631_g_at', '32604_s_at', '31956_f_at', '33328_at', '40570_at',
       '31432_g_at', '1664_at', '31538_at', '40004_at', '31335_at',
       '33404_at', '1296_at', '39050_at', '1199_at', '37667_at',
       '36897_at', '1367_f_at', '40607_at', '38809_s_at', '38277_at',
       '31523_f_at', '31957_r_at', '32334_f_at', '37749_at', '38354_at',
       '824_at', '39327_at', '38183_at', '41556_s_at', '37470_at'],
      dtype=object)

In [41]:
#Xvariance
X = data[data.Class == 0]
Y = data[data.Class == 1]
X_=X[:].apply(pd.to_numeric, errors='coerce')
#X_var.dtypes
X_var = (X_).var()
#print (X_var)
Y_ = Y[:].apply(pd.to_numeric, errors='coerce')
Y_var=Y_.var()
D = X_var + Y_var
DL=D.nlargest(n=50)
ft4=np.array(DL.index)
print (ft4)
#print (DL)

['41238_s_at' '35905_s_at' '31962_at' '35119_at' '39798_at' '38126_at'
 '31509_at' '34593_g_at' '34407_at' '40887_g_at' '32438_at' '32466_at'
 '31505_at' '1288_s_at' '32744_at' '32796_f_at' '33994_g_at' '31952_at'
 '32052_at' '35981_at' '33657_at' '256_s_at' '327_f_at' '38751_i_at'
 '34645_at' '38418_at' '34085_at' '347_s_at' '35278_at' '1366_i_at'
 '31957_r_at' '31511_at' '2020_at' '31545_at' '36333_at' '32395_r_at'
 '34592_at' '37032_at' '39026_r_at' '32748_at' '31961_r_at' '32330_at'
 '33676_at' '769_s_at' '32315_at' '41152_f_at' '41231_f_at' '31444_s_at'
 '32436_at' '31687_f_at']


In [43]:
#Correlation
df1 = data.drop(['Class'], axis=1)
op=data['Class']
a5=[]
c5=[]
print (op.dtype) 

for i in df1.columns.values:
    #print (df1[i].dtype) 
    df1[i]=df1[i].astype('float64')
    correlation=op.corr(df1[i])
    if correlation>0.5:
        a5.append(correlation)
        c5.append(i)

dfc5=pd.DataFrame(c5)
dfa5=pd.DataFrame(a5)
info5 = pd.concat([dfc5,dfa5], axis=1)
info5.columns = ['name','correlation']
info5

int64


,name,correlation
0,31596_f_at,0.542286
1,34653_at,0.511708
2,32959_at,0.545990
3,35956_s_at,0.520279
4,36308_at,0.667173
...,...,...
111,484_at,0.566485
112,459_s_at,0.549988
113,404_at,0.573566
114,288_s_at,0.523315


In [44]:
top5 = info5.nlargest(50,'correlation')
ft5 = np.array(top5['name'])
ft5

array(['40570_at', '35321_at', '35330_at', '39350_at', '39690_at',
       '38486_at', '36943_r_at', '1664_at', '36308_at', '39542_at',
       '39063_at', '40004_at', '649_s_at', '37749_at', '36118_at',
       '33192_g_at', '41289_at', '1296_at', '41470_at', '35312_at',
       '37605_at', '38031_at', '820_at', '34408_at', '33911_at',
       '33405_at', '36113_s_at', '33929_at', '38713_at', '39327_at',
       '2057_g_at', '659_g_at', '35824_at', '35648_at', '652_g_at',
       '404_at', '35832_at', '35850_at', '32845_at', '41686_s_at',
       '38809_s_at', '39084_at', '37735_at', '484_at', '33404_at',
       '260_at', '36596_r_at', '36114_r_at', '37210_at', '41836_at'],
      dtype=object)

In [45]:
from functools import reduce
def top_fs_union(n):
    f1 = ft1[:n]
    f2 = ft2[:n]
    f3 = ft3[:n]
    f4 = ft4[:n]
    f5 = ft5[:n]
    return(reduce(np.union1d, (f1,f2,f3,f4,f5)))

In [46]:
def assemble_run(n,classifier,test_size):
  top_fs=top_fs_union(n)
  top_fs=np.append(top_fs,['Class'], axis=0)
  dfs = data.loc[:,top_fs]
  X = dfs.drop(['Class'], axis=1)
  y = dfs['Class']
  from sklearn.model_selection import train_test_split
  x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=test_size)
  model = classifier()#modify the code ,keep classifier in different function and call here
  model.fit(x_train, y_train)
  print (model.score(x_test,y_test))

In [47]:
#SUPPORT VECTOR MACHINE
#Feature Subset by Union of 5 Best Features from each
SVM5  = assemble_run(5,SVC,.5)
#Feature Subset by Union of 10 Best Features from each
SVM10 = assemble_run(10,SVC,.5)
#Feature Subset by Union of 20 Best Features from each
SVM20 = assemble_run(20,SVC,.5)
#Feature Subset by Union of 30 Best Features from each
SVM30 = assemble_run(30,SVC,.5)
#Feature Subset by Union of 40 Best Features from each
SVM40 = assemble_run(40,SVC,.5)
#Feature Subset by Union of 50 Best Features from each
SVM50 = assemble_run(50,SVC,.5)

0.3333333333333333
0.25
0.5
0.5
0.3333333333333333
0.5833333333333334


In [48]:
#K-NEAREST NEIGHBOUR
#Feature Subset by Union of 5 Best Features from each
KNN5 = assemble_run(5,KNeighborsClassifier,.5)
#Feature Subset by Union of 10 Best Features from each
KNN10 = assemble_run(10,KNeighborsClassifier,.5)
#Feature Subset by Union of 20 Best Features from each
KNN20 = assemble_run(20,KNeighborsClassifier,.5)
#Feature Subset by Union of 30 Best Features from each
KNN30 = assemble_run(30,KNeighborsClassifier,.5)
#Feature Subset by Union of 40 Best Features from each
KNN40 = assemble_run(40,KNeighborsClassifier,.5)
#Feature Subset by Union of 50 Best Features from each
KNN50 = assemble_run(50,KNeighborsClassifier,.5)

0.5
0.25
0.5833333333333334
0.3333333333333333
0.8333333333333334
0.3333333333333333


In [49]:
#NAIVE BAYES
#Feature Subset by Union of 5 Best Features from each
NB5  = assemble_run(5,GaussianNB,.5)
#Feature Subset by Union of 10 Best Features from each
NB10 = assemble_run(10,GaussianNB,.5)
#Feature Subset by Union of 20 Best Features from each
NB20 = assemble_run(20,GaussianNB,.5)
#Feature Subset by Union of 30 Best Features from each
NB30 = assemble_run(30,GaussianNB,.5)
#Feature Subset by Union of 40 Best Features from each
NB40 = assemble_run(40,GaussianNB,.5)
#Feature Subset by Union of 50 Best Features from each
NB50 = assemble_run(50,GaussianNB,.5)

0.6666666666666666
0.5833333333333334
1.0
0.75
0.75
0.6666666666666666


In [50]:
#DECISION TREE
#Feature Subset by Union of 5 Best Features from each
DT5  = assemble_run(5,DecisionTreeClassifier,.5)
#Feature Subset by Union of 10 Best Features from each
DT10 = assemble_run(10,DecisionTreeClassifier,.5)
#Feature Subset by Union of 20 Best Features from each
DT20 = assemble_run(20,DecisionTreeClassifier,.5)
#Feature Subset by Union of 30 Best Features from each
DT30 = assemble_run(30,DecisionTreeClassifier,.5)
#Feature Subset by Union of 40 Best Features from each
DT40 = assemble_run(40,DecisionTreeClassifier,.5)
#Feature Subset by Union of 50 Best Features from each
DT50 = assemble_run(50,DecisionTreeClassifier,.5)

0.8333333333333334
0.8333333333333334
0.6666666666666666
0.75
0.75
0.8333333333333334
